In [1]:
import pandas as pd
import numpy as np
import nltk
# nltk.download()
nltk.download('punkt')
#Downloading stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bhartibulchandani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhartibulchandani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
true_df = pd.read_csv("True.csv")

In [3]:
fake_df = pd.read_csv("Fake.csv")

In [4]:
true_df.tail()

,title,text,subject,date
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


In [5]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
true_df["subject"].value_counts()

politicsNews    11272
worldnews       10145
Name: subject, dtype: int64

In [7]:
fake_df["subject"].value_counts()

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

In [8]:
true_df.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [9]:
fake_df.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [10]:
true_df["label"] = 0
fake_df["label"] = 1

In [11]:
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [12]:
fake_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [13]:
df = pd.concat([true_df, fake_df]).reset_index(drop=True)

In [14]:
df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [15]:
# fake = 1 ; real = 0
df["label"].value_counts()

1    23481
0    21417
Name: label, dtype: int64

In [16]:
df.shape

(44898, 5)

In [17]:
df_copy = df.copy()

In [18]:
df_copy.shape

(44898, 5)

In [19]:
df_copy["subject"].value_counts()

politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: subject, dtype: int64

In [20]:
# corpus = df_copy["text"].values

In [21]:
# corpus

In [22]:
df_copy["news"] = df_copy["title"] + ' ' + df_copy["text"]
df_copy.head()

,title,text,subject,date,label,news
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,"As U.S. budget fight looms, Republicans flip t..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,U.S. military to accept transgender recruits o...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,Senior U.S. Republican senator: 'Let Mr. Muell...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,FBI Russia probe helped by Australian diplomat...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,Trump wants Postal Service to charge 'much mor...


In [23]:
# function to preprocess data in column data

def data_preprocess(text):
    relevant_text = []
    for token in simple_preprocess(text):
        if (token not in STOPWORDS) and (token not in stopwords.words('english')):
            relevant_text.append(token)
    return relevant_text

In [24]:
df_copy["relevant_news"] = df_copy["news"].apply(data_preprocess)

In [25]:
df_copy["relevant_news"][0]

['budget',
 'fight',
 'looms',
 'republicans',
 'flip',
 'fiscal',
 'script',
 'washington',
 'reuters',
 'head',
 'conservative',
 'republican',
 'faction',
 'congress',
 'voted',
 'month',
 'huge',
 'expansion',
 'national',
 'debt',
 'pay',
 'tax',
 'cuts',
 'called',
 'fiscal',
 'conservative',
 'sunday',
 'urged',
 'budget',
 'restraint',
 'keeping',
 'sharp',
 'pivot',
 'way',
 'republicans',
 'representative',
 'mark',
 'meadows',
 'speaking',
 'cbs',
 'face',
 'nation',
 'drew',
 'hard',
 'line',
 'federal',
 'spending',
 'lawmakers',
 'bracing',
 'battle',
 'january',
 'return',
 'holidays',
 'wednesday',
 'lawmakers',
 'begin',
 'trying',
 'pass',
 'federal',
 'budget',
 'fight',
 'likely',
 'linked',
 'issues',
 'immigration',
 'policy',
 'november',
 'congressional',
 'election',
 'campaigns',
 'approach',
 'republicans',
 'seek',
 'control',
 'congress',
 'president',
 'donald',
 'trump',
 'republicans',
 'want',
 'big',
 'budget',
 'increase',
 'military',
 'spending',
 '

In [26]:
df_copy.head()

,title,text,subject,date,label,news,relevant_news
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,"As U.S. budget fight looms, Republicans flip t...","[budget, fight, looms, republicans, flip, fisc..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,U.S. military to accept transgender recruits o...,"[military, accept, transgender, recruits, mond..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,Senior U.S. Republican senator: 'Let Mr. Muell...,"[senior, republican, senator, let, mr, mueller..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,FBI Russia probe helped by Australian diplomat...,"[fbi, russia, probe, helped, australian, diplo..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,Trump wants Postal Service to charge 'much mor...,"[trump, wants, postal, service, charge, amazon..."


In [27]:
unique_words = []
for i in df_copy['relevant_news']:
    for j in i:
        unique_words.append(j)

In [28]:
len(unique_words)

9879257

In [29]:
updated_unique_words = list(set(unique_words))

In [30]:
len(updated_unique_words)

114597

In [31]:
df_copy["updated_relevant_news"] = df_copy["relevant_news"].apply(lambda x: " ".join(x))

In [32]:
df_copy.head()

,title,text,subject,date,label,news,relevant_news,updated_relevant_news
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,"As U.S. budget fight looms, Republicans flip t...","[budget, fight, looms, republicans, flip, fisc...",budget fight looms republicans flip fiscal scr...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,U.S. military to accept transgender recruits o...,"[military, accept, transgender, recruits, mond...",military accept transgender recruits monday pe...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,Senior U.S. Republican senator: 'Let Mr. Muell...,"[senior, republican, senator, let, mr, mueller...",senior republican senator let mr mueller job w...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,FBI Russia probe helped by Australian diplomat...,"[fbi, russia, probe, helped, australian, diplo...",fbi russia probe helped australian diplomat ti...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,Trump wants Postal Service to charge 'much mor...,"[trump, wants, postal, service, charge, amazon...",trump wants postal service charge amazon shipm...


In [33]:
df_copy.sample(5)

,title,text,subject,date,label,news,relevant_news,updated_relevant_news
16912,Merkel says Britain must move on Brexit bill t...,BRUSSELS (Reuters) - German Chancellor Angela ...,worldnews,"October 20, 2017",0,Merkel says Britain must move on Brexit bill t...,"[merkel, says, britain, brexit, break, deadloc...",merkel says britain brexit break deadlock brus...
38905,“NEVER-TRUMPER” and Weekly Standard Editor Sen...,The former Fox News contributor and editor of ...,left-news,"Dec 22, 2017",1,“NEVER-TRUMPER” and Weekly Standard Editor Sen...,"[trumper, weekly, standard, editor, sends, vil...",trumper weekly standard editor sends vile twee...
17869,Malaysia's royals call for religious tolerance...,KUALA LUMPUR (Reuters) - Malaysia s royals mad...,worldnews,"October 10, 2017",0,Malaysia's royals call for religious tolerance...,"[malaysia, royals, religious, tolerance, rare,...",malaysia royals religious tolerance rare publi...
2787,Conservative groups urge lawmakers to gut clim...,WASHINGTON (Reuters) - A coalition of 14 conse...,politicsNews,"July 12, 2017",0,Conservative groups urge lawmakers to gut clim...,"[conservative, groups, urge, lawmakers, gut, c...",conservative groups urge lawmakers gut climate...
24071,WATCH: Senate Dems Hold 24-Hour Debate Marath...,"Senate Democrats had one last, huge move plann...",News,"February 6, 2017",1,WATCH: Senate Dems Hold 24-Hour Debate Marath...,"[watch, senate, dems, hold, hour, debate, mara...",watch senate dems hold hour debate marathon st...


In [63]:
# wc_true = WordCloud().generate(" ".join(df_copy[df_copy["label"] == 0]))
# plt.imshow(wc_true, interpolation='bilinear')

In [34]:
# data = []
# for sentences in corpus:
#     sentence = sent_tokenize(sentences)
#     data.append(sentence)

In [35]:
# data

In [36]:
# from nltk.corpus import stopwords
# stopwords.words('english')

In [37]:
# import re
# sentence = []
# for i in data:
#     print(i[0])
#     for j in i:
#         sentence.append(j)
# print(sentence)

In [38]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vector = TfidfVectorizer()
# X = vector.fit_transform(corpus)

In [39]:
# vector.get_feature_names()

In [40]:
# X.toarray()

In [41]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import confusion_matrix, accuracy_score

In [42]:
# y = df_copy.pop("label")

In [43]:
# df_copy.head()

In [44]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df_copy, y, test_size=0.25, random_state=11)

In [45]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=15)

In [46]:
# X_train.values.shape

In [47]:
# y_train.shape

In [48]:
# build model to Naive Bayes classifier

# model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# model.fit(X_train.reshape(-1, 1), y_train)

# predict = model.predict(X_val)

In [49]:
# model.fit(X_train.reshape(-1, 1), y_train)